In [44]:
import os
from datetime import datetime
from shutil import copyfile
import pandas as pd
from datetime import datetime
import numpy as np

In [45]:
def get_date(raw_file_name):
    return datetime.strptime(raw_file_name.split('covid19-')[1].replace('.csv', ''), '%Y-%m-%d')

In [46]:
def convert_back_to_str_file(date):
    return  'mexico-covid19-' + date.isoformat().split('T')[0] + '.csv'

In [70]:
def fix_dateformat(date_string):
    if pd.isna(date_string): return ''
    try:
        date = datetime.strptime(date_string, '%d-%m-%Y')
        if (datetime.now() - date).days < 0:
            raise
        return date.isoformat().split('T')[0]
    except:
        try:
            date = datetime.strptime(date_string, '%m-%d-%Y')
            return date.isoformat().split('T')[0]
        except:
            None # print(date_string) should be just nan
    return ''

In [73]:
def switch_date(date_string):
    date_str_array = date_string.split('-')
    return date_str_array[0]+'-'+date_str_array[2]+'-'+date_str_array[1]

In [82]:
def fix_row_details(row_o):
    row = row_o.to_dict()
    if row['Date_Arrival'] == '' or row['Date_Confirmed']=='' or row['Date_Symptoms'] == '': return row
    date_arrival = datetime.strptime(row['Date_Arrival'], '%Y-%m-%d')
    date_confirmed = datetime.strptime(row['Date_Confirmed'], '%Y-%m-%d')
    date_symptoms = datetime.strptime(row['Date_Symptoms'], '%Y-%m-%d')
    if date_arrival > date_confirmed and date_arrival < datetime.now():
        row['Date_Arrival'] = switch_date(row['Date_Arrival'])
    return row

In [85]:
available_files = [file for file in os.listdir() if 'mexico' in file] 
max_date = max([get_date(file) for file in available_files])
max_date_file = convert_back_to_str_file(max_date)
df_data = pd.read_csv(max_date_file)
for column_name in ['Date_Arrival', 'Date_Confirmed','Date_Symptoms']:
    df_data[column_name] = df_data[column_name].apply(lambda x: fix_dateformat(x))
df_data = pd.DataFrame([fix_row_details(row) for i, row in df_data.iterrows()])
df_data.to_csv('latest.csv', index=None)